## Loop to get the Spatial Structure Parameters

In [1]:
using PyCall

using Distributions, Statistics, Distances
using DataFrames
using LightGraphs, SimpleWeightedGraphs
using Random , Printf, Glob

import PyPlot , CSV
import Distances, StatsBase

rootdir = "/home/stephane/Science/cluster/GAIA"

push!(LOAD_PATH,"$rootdir/master/src")
using GaiaClustering

## directory
wdir    = "$rootdir/products"
plotdir = "$rootdir/products/plotdens"
ocdir= "$wdir/oc2"

cd(wdir)

In [ ]:
cd(ocdir)
files= glob("*-oc.csv")
cd(wdir)

df = DataFrame(voname= String[], C= Float64[], Cerr= Float64[], s= Float64[], serr= Float64[], m= Float64[],
merr= Float64[])

fitflag= true
noc= 0 ; nfail= 0

for f in files
    if fitflag
        fname= f[1:end-4] ; fvot= f[1:end-7]
        oc= CSV.read("$ocdir/$fname.csv" , delim= ";")
        ocfile= "$ocdir/$fname.csv" ; voname= "$fvot.vot"
        println("## ocfile: $voname")
        println("## fitting the radial profile....")
    
        ntest= 100 ; niter= 30000 ; nburnin= 5000
    
        ## estimating the binning
        nstar= length(oc.X)
        println("## Nstar: $nstar")
        nbin= min(trunc(Int, nstar/10),20)
        println("## NBIN: $nbin")
    
        verbose= false
        θfit, θfiterr, fitfound= spatialParameter(ocfile, ntest, nbin, niter, verbose, nburnin)
    
        println("## Fit:")
        println(θfit)
        println(θfiterr)
        println(" ")
    
        push!(df, [voname,θfit.C , θfiterr.C, θfit.s , θfiterr.s, θfit.m , θfiterr.m])
        
        noc += 1
        if !fitfound
            println("################################ Failing...########### ")
            println("###################################################### ")
            nfail += 1
        end
        
        println("## OCs analyzed: $noc")
        println("## Fit failed: $nfail \n\n")
    end
end

if fitflag 
    CSV.write("fitCauchy-p100-30k.csv", df, delim=";") 
else
    
    println("\n##\n## No fit...\n##")
end

## ocfile: ASCC 10-2.7deg.vot
## fitting the radial profile....
## Nstar: 119
## NBIN: 11
[1.7915301463051434, 3.981178102900319, 3.463624949523278, 2.132773983696599, 0.5971767154350477, 0.6514655077473251, 0.3215566929265639, 0.15924712411601283, 0.14051216833765826, 0.09429106033184965, 0.08531095934786395]
## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(4.1201324040427645, 4.0901051823660435, 2.9414236420130324)
GaiaClustering.modelCauchy(0.03775146349018574, 0.06236692807150529, 0.05730389750078035)
 
## OCs analyzed: 1
## Fit failed: 0 


## ocfile: ASCC 100-1.8deg.vot
## fitting the radial profile....
## Nstar: 149
## NBIN: 14
[0.0, 1.3361114218789711, 1.0203032676166692, 0.36439402414881034, 0.48585869886508054, 0.5631544009572524, 0.39242433369871876, 0.26722228437579426, 0.3429590815518214, 0.30685812559899833, 0.1908730602684244, 0.09505931064751581, 0.1603333706254767, 0.040488224905423316]
## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(0.9572900582372619, 6.

## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(2.224999749361035, 23.007441089309264, 6.45221152917268)
GaiaClustering.modelCauchy(0.03932367260503647, 0.7382207609608069, 0.3321816265386814)
 
## OCs analyzed: 15
## Fit failed: 0 


## ocfile: ASCC 127-3.4deg.vot
## fitting the radial profile....
## Nstar: 171
## NBIN: 17
[4.265361107213402, 2.258132350877683, 1.103975815984645, 0.6810240423281904, 0.3345381260559531, 0.41056951834139677, 0.23160331803873663, 0.13381525042238132, 0.1771084196766808, 0.1452599757874535, 0.0, 0.010908851936607166, 0.08028915025342882, 0.009292725723776457, 0.0, 0.016187328680126747, 0.007603139228544383]
## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(3.3821978254443334, 3.1550395194679375, 1.6213024293157159)
GaiaClustering.modelCauchy(0.06310714915198885, 0.05392742376547812, 0.014503010667039289)
 
## OCs analyzed: 16
## Fit failed: 0 


## ocfile: ASCC 16-3.3deg.vot
## fitting the radial profile....
## Nstar: 520
## NBIN: 20
[0.503056

## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(1.6017611848078084, 10.892273357050003, 2.3056886879664926)
GaiaClustering.modelCauchy(0.039370182747805994, 0.985354726391336, 0.2585046664123527)
 
## OCs analyzed: 28
## Fit failed: 0 


## ocfile: ASCC 33-2.6deg.vot
## fitting the radial profile....
## Nstar: 701
## NBIN: 20
[1.7355080071548448, 2.2778542593907334, 1.3667125556344406, 1.1466749332987365, 0.6267112248059159, 0.6409546617333239, 0.5840651947155726, 0.3687954515204044, 0.4785408107963724, 0.26831867215880817, 0.22726890569884858, 0.11790135918171527, 0.0997917104114036, 0.04017379646191768, 0.022441913885622982, 0.02099404847364731, 0.01643473491623907, 0.009297364324043798, 0.005863202726874487, 0.002781262831978911]
## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(1.590503131410761, 21.392414133978402, 4.446358486209549)
GaiaClustering.modelCauchy(0.03130653845998726, 1.042289837764552, 0.32053135693729523)
 
## OCs analyzed: 29
## Fit failed: 0 


## ocfi

## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(18.11851759346721, 8.076722844881019, 6.355290448463081)
GaiaClustering.modelCauchy(0.3673106387969353, 0.4074793969928094, 0.5423665150464724)
 
## OCs analyzed: 41
## Fit failed: 0 


## ocfile: ASCC 84-1.0deg.vot
## fitting the radial profile....
## Nstar: 149
## NBIN: 14
[1.3729335687183455, 1.8305780916244605, 1.9907536746416012, 1.9613336695976364, 0.5720556536326439, 0.4056394634849659, 0.23762311766279057, 0.16017558301714033, 0.10095099769987835, 0.03612983075574592, 0.06537778898658789, 0.029846381928659703, 0.027458671374366913, 0.012712347858503196]
## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(2.3621039119331404, 6.959581796043844, 3.682919203992285)
GaiaClustering.modelCauchy(0.02013625833683077, 0.09237587696901292, 0.06268925651462351)
 
## OCs analyzed: 42
## Fit failed: 0 


## ocfile: ASCC 89-3.8deg.vot
## fitting the radial profile....
## Nstar: 85
## NBIN: 8
[1.4809835057452858, 1.6661064439634465, 1.0

## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(19.705733767802503, 2.289530906212587, 2.0159939197130634)
GaiaClustering.modelCauchy(0.37669789621001476, 0.07054252451096253, 0.04618282391687185)
 
## OCs analyzed: 54
## Fit failed: 0 


## ocfile: Alessi 62-1.4deg.vot
## fitting the radial profile....
## Nstar: 92
## NBIN: 9
[4.0920406836033365, 1.4185741036491568, 0.4910448820324006, 0.2104478065853144, 0.2000553223094965, 0.05952059176150311, 0.012590894411087199, 0.0, 0.009628331020243144]
## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(4.509992472646941, 2.2884690055822663, 1.8161064214212332)
GaiaClustering.modelCauchy(0.056246337705713634, 0.03392217769781125, 0.0195179954837677)
 
## OCs analyzed: 55
## Fit failed: 0 


## ocfile: Alessi 9-4.5deg.vot
## fitting the radial profile....
## Nstar: 351
## NBIN: 20
[16.38271482727875, 9.89789020814758, 7.3722216722754395, 4.388227185878237, 4.436985265721326, 3.6302606719538164, 2.8354698739520914, 2.3891459123114847, 

...

## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(2.034357936366238, 5.257224002873489, 2.8258079494877677)
GaiaClustering.modelCauchy(0.019337174029266206, 0.06885563187466646, 0.039563146526094266)
 
## OCs analyzed: 81
## Fit failed: 0 


## ocfile: Collinder 463-2.5deg.vot
## fitting the radial profile....
## Nstar: 189
## NBIN: 18
[1.6552184530253664, 1.517283581939919, 1.737979375676635, 1.1231839502672127, 1.0115223879599455, 0.6018976192819521, 0.6366224819328331, 0.4138046132563415, 0.21907303054747493, 0.32668785257079563, 0.1773448342527181, 0.12594053446932144, 0.14896966077228288, 0.06130438714908765, 0.05707649838018495, 0.013348535911494907, 0.025079067470081314, 0.011822988950181182]
## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(1.9103338893577082, 8.309150986462956, 3.193726963061661)
GaiaClustering.modelCauchy(0.043759677376246114, 0.4813143197688794, 0.22430998937095864)
 
## OCs analyzed: 82
## Fit failed: 0 


## ocfile: Collinder 65-7.5deg.vot
## fit

[30.375065699003, 24.09056934748514, 21.157804383443477, 23.79130761646047, 21.76297810618224, 22.852714005519502, 13.213556431396517, 8.239672994212311, 3.758375269044998, 2.094832117172622, 1.8953242964895127, 1.8671329740016853, 1.3825891973339286, 0.6206909976807757, 0.541766926854989, 0.5406018366897082, 0.28565892506899343, 0.14963086551233037, 0.05661708424790858, 0.05371364403006735]
## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(33.346532085799225, 7.636657634070877, 5.894122719498406)
GaiaClustering.modelCauchy(0.2361341288926396, 0.10304192462093002, 0.11487249929059143)
 
## OCs analyzed: 95
## Fit failed: 0 


## ocfile: ESO 128-16-1.0deg.vot
## fitting the radial profile....
## Nstar: 177
## NBIN: 17
[2.3080820762437044, 2.0516285122166265, 1.8464656609949626, 2.527899416838344, 2.308082076243705, 1.5387213841624678, 1.4203582007653581, 1.2822678201353912, 0.36205209039116915, 0.0809853360085511, 0.2198173405946383, 0.36795511360406835, 0.06154885536649874, 0.0569

## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(3.3420977697920193, 5.543722592752733, 3.1058648836554674)
GaiaClustering.modelCauchy(0.08220908864637118, 0.4693252237158153, 0.4041801059848961)
 
## OCs analyzed: 109
## Fit failed: 0 


## ocfile: FSR 0222-1.0deg.vot
## fitting the radial profile....
## Nstar: 162
## NBIN: 16
[1.3153799604596919, 2.1922999340994864, 1.0523039683677533, 1.5659285243567762, 0.8769199736397946, 0.9167799724416033, 0.6745538258767649, 0.43845998681989745, 0.051583527861164394, 0.18461473129258832, 0.25054856389708424, 0.07625391075128644, 0.0350767989455918, 0.016239258771107296, 0.015119309890341286, 0.014143870542577342]
## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(2.172821417749974, 6.889614250830916, 3.402225388241131)
GaiaClustering.modelCauchy(0.046379212468178076, 0.42413386302114686, 0.29047996261195813)
 
## OCs analyzed: 110
## Fit failed: 0 


## ocfile: FSR 0227-1.0deg.vot
## fitting the radial profile....
## Nstar: 158
## NBI

## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(3.825943795180475, 4.0387996941299935, 3.5796155578216404)
GaiaClustering.modelCauchy(0.12093583445739774, 0.36062733965091803, 0.46325467060149605)
 
## OCs analyzed: 124
## Fit failed: 0 


## ocfile: FSR 0581-1.0deg.vot
## fitting the radial profile....
## Nstar: 127
## NBIN: 12
[3.4774120335527403, 3.2842224761331433, 2.5501021579386767, 2.401070213643558, 1.2879303827973114, 0.684944794487661, 0.3120754389085793, 0.19318955741959662, 0.13636909935500943, 0.06100722865882002, 0.0, 0.050397275848590474]
## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(4.650557275219921, 4.292509889002369, 3.1066299965940103)
GaiaClustering.modelCauchy(0.11145604973440784, 0.3540601919985945, 0.3971769810150514)
 
## OCs analyzed: 125
## Fit failed: 0 


## ocfile: FSR 0596-1.0deg.vot
## fitting the radial profile....
## Nstar: 74
## NBIN: 7
[0.7871367936982558, 1.0495157249310079, 0.7346610074517057, 0.8246194981600774, 0.29153214581416903

## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(2.3298710038246058, 5.403408746990158, 3.4368246815119323)
GaiaClustering.modelCauchy(0.07710660218967208, 0.4069196785516014, 0.5287495507660143)
 
## OCs analyzed: 140
## Fit failed: 0 


## ocfile: FSR 1158-1.0deg.vot
## fitting the radial profile....
## Nstar: 49
## NBIN: 4
[0.8349240899035175, 0.19879144997702797, 0.06361326399264895, 0.02839877856814686]
## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(1.1345113828218554, 2.7285488272358087, 1.401662168602894)
GaiaClustering.modelCauchy(0.023863399968828587, 0.06921675361894526, 0.02438594932645775)
 
## OCs analyzed: 141
## Fit failed: 0 


## ocfile: FSR 1160-1.0deg.vot
## fitting the radial profile....
## Nstar: 80
## NBIN: 8
[0.8334842981558228, 0.8057014882172954, 0.20003623155739747, 0.1547899410860814, 0.09260936646175807, 0.03788564991617377, 0.0, 0.005556561987705488]
## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(0.9952687341625072, 5.919027469398705, 

In [ ]:
csvfile= "test-fitCauchy3.csv"
df= CSV.read(csvfile , delim= ";")


fig= PyPlot.figure(figsize=(10,10))

ax= PyPlot.subplot(321)
PyPlot.grid("on")
ax.set_xscale("log")
ax.set_yscale("log")
PyPlot.xlabel("C")
PyPlot.ylabel("s")
PyPlot.scatter(df.C, df.s , s=4, facecolor="blue" )
PyPlot.errorbar(df.C, df.s, xerr=df.Cerr, yerr=df.serr, linewidth=0.5,ls="none")


ax= PyPlot.subplot(322)
PyPlot.grid("on")
ax.set_xscale("log")
#ax.set_yscale("log")
PyPlot.xlabel("s")
PyPlot.ylabel("m")
PyPlot.scatter(df.s, df.m , s=4, facecolor="red" )
PyPlot.errorbar(df.s, df.m, xerr=df.serr, yerr=df.merr, linewidth=0.5,ls="none")

ax = PyPlot.subplot(323)
PyPlot.grid("on")
nbins = 100
h = PyPlot.hist(df.C,nbins,range = [0,30], color = "k", alpha=0.6 , label = "C")
PyPlot.xlabel("C")

ax = PyPlot.subplot(324)
PyPlot.grid("on")
nbins = 100
h = PyPlot.hist(df.s,nbins,range = [0,30], color = "r", alpha=0.6 , label = "s")
PyPlot.xlabel("s")

ax = PyPlot.subplot(325)
PyPlot.grid("on")
nbins = 100
h = PyPlot.hist(df.m,nbins,range = [0,10], color = "g", alpha=0.6 , label = "s")
PyPlot.xlabel("m")

PyPlot.show()

--------------
## Checking all the fit with data.

Figs saved with data density with Cauchy Fit.

In [ ]:
function _plotFitDensityFig(filelist, θlist, ifig, nrow, ncol, ocdir)
    
    fig= PyPlot.figure(figsize=(15,10))
    
    for i in 1:length(filelist)
        ocname= filelist[i][1:end-4]
        θ= θlist[i]
        ocfile= "$ocdir/$ocname-oc.csv"
        _plotDensity(ocfile, θ,nrow, ncol, i, ocname)
    end
    
    figname= "densityOC-$ifig.png"
    PyPlot.savefig(figname)
    PyPlot.close(fig)
end


function _plotDensity(ocfile, θ, irow, icol, nplot, title)
    oc= CSV.read(ocfile, delim= ";")
    # binned density
    nstar= length(oc.X)
    nbin= min(trunc(Int, nstar/10),20)
    r2d,ρ2d,err2d= density2D(oc.Y, oc.Z, nbin)
    ρ2dfit= model_rad(r2d, θ, fdens1)
    
    ax= PyPlot.subplot(irow, icol, nplot)
    ax.set_xlabel("r [pc]")
    ax.set_ylabel("ρ")
    ax.set_yscale("log")
    ax.set_xscale("log")
    ax.set_xlim(r2d[1]*0.9, r2d[end]*1.1)
    ax.set_ylim(minimum(ρ2d[ρ2d .> 0])*0.3,maximum(ρ2d)*1.5)
    PyPlot.grid("on")
    PyPlot.scatter(r2d, ρ2d , s=4, facecolor="blue" )
    PyPlot.errorbar(r2d, ρ2d, yerr=2 .* err2d, linewidth=0.5)
    PyPlot.plot(r2d, ρ2dfit, "k-", linewidth=1)
    
    ## TEXT
    ax.text(0.02, 0.01, title, verticalalignment= "bottom" , horizontalalignment="left",
        transform=ax.transAxes, color="green", fontsize=15)
    
    val=θ.C ; txt= @sprintf("C= %3.3f",val)
    ax.text(0.02, 0.2, txt, verticalalignment= "bottom" , horizontalalignment="left",
        transform=ax.transAxes, color="red", fontsize=12)
    val=θ.s ; txt= @sprintf("s= %3.3f [pc]",val)
    ax.text(0.02, 0.3, txt, verticalalignment= "bottom" , horizontalalignment="left",
        transform=ax.transAxes, color="red", fontsize=12) 
    val=θ.m ; txt= @sprintf("m= %3.3f",val)
    ax.text(0.02, 0.4, txt, verticalalignment= "bottom" , horizontalalignment="left",
        transform=ax.transAxes, color="red", fontsize=12)
    
    val= nstar ; txt= @sprintf("Nstar= %d",val)
    ax.text(0.02, 0.52, txt, verticalalignment= "bottom" , horizontalalignment="left",
        transform=ax.transAxes, color="blue", fontsize=12)
    
end

In [ ]:
cd(plotdir)

rowperfig= 4
colperfig= 3
plotperfig= rowperfig*colperfig

ifig= 1
plotaccum=0
filevot= [] ; θlist= [] ; θerrlist= []
for item in 1:length(df.voname)
    push!(filevot, df.voname[item])
    θ= GaiaClustering.modelCauchy(df.C[item], df.s[item], df.m[item])
    push!(θlist, θ)
    θerr= GaiaClustering.modelCauchy(df.Cerr[item], df.serr[item], df.merr[item])
    push!(θerrlist, θerr)
    plotaccum += 1
    
    if plotaccum == plotperfig
        _plotFitDensityFig(filevot, θlist, ifig,rowperfig, colperfig , ocdir)
        println("## Fig $ifig done.")
        filevot= [] ; θlist= [] ; θerrlist= []
        plotaccum= 0 ; ifig += 1

    end
end

cd(wdir)